In the name of Almighty

# HW 2

## Meshkat Ahmadi

# Q1) Two players playing with a row of N elements (v_1, v_2, .. , v_n) 

### Solution 1.1) 

brute-force solution to this problem is generating all possible solutions for the player 1 and pick the one with the highest score.
We can generate all of possible solutions using a recursive function returning the paths and choose always the maximum score.


### Solution 1.2) 

To make it more efficient we can use a dynamic programming approach.
We can consider 'findmax' array storying the maximum score for player 1 when when the remaining head and tails are at i and j.

In this case, player 1 can choose either i or j:

1) choosing i: the player 2 chooses either the element $v_{i+1}$ or $v_j$. In this case the player 1 will collect the  score of $v_i$ + min(findmax[i+2, j], findmax[i+1, j-1] )

2) choosing j: the player 2 chooses either the element $v_i$ or $v_{j-1}$. In this case the player 1 will collect the  score of $v_j$ + min(findmax[i+1, j-1], findmax[i, j-2] )

The player 1 has to find the max between these two options. So we can try to fill out the table n*n and the results would be at findmax(0,n-1) if we are zero-based.

### Solution 1.3)

Since we need to fill out the table n*n the time and space complexity similar to dynamic progrmming cases would be in the O($n^2$)

### Solution 1.4)

In [252]:
import numpy as np


def find_max(Vs):
    n = len(Vs)
    # In this table we store solutions of the subproblems
    table = [[0] * n for _ in range(n)]
    path = [['']* n for _ in range(n)]
    # Fill table using the recursive formula diagonal as i<j
    for gap in range(n):
        for i in range(n - gap):
            j = i + gap
            # if the player 1 chooses i and player 2 chooses i+1
            s_i1 = table[i + 2][j] if (i + 2) <= j else 0
            # if the player 1 chooses i and player 2 chooses j
            # or if the player 1 chooses j and player 2 chooses i
            s_i2_j1 = table[i + 1][j - 1] if (i + 1) <= (j - 1) else 0
            # if the player 1 chooses j and player 2 chooses j-1
            s_j2 = table[i][j - 2] if i <= (j - 2) else 0
            
            # calculating the max score
            table[i][j] = max(Vs[i] + min(s_i1, s_i2_j1), Vs[j] + min(s_i2_j1,s_j2))
            
            if j>i:
                # storing the path            
                if Vs[i] + min(s_i1, s_i2_j1) > Vs[j] + min(s_i2_j1,s_j2):            
                    # choosing i1 means player 1 chooses i and player 2 chooses i+1
                    # i.e. another L
                    if s_i1 < s_i2_j1:
                        path[i][j]= path[i+2][j] + 'LR'
                    else:
                        # player 2 has chosen R
                        path[i][j]= path[i+1][j-1] + 'LL'
                else:
                    # choosing the jth means Right
                    if  s_i2_j1 < s_j2:
                        # player 2 has chosen R                    
                        path[i][j]=  path[i+1][j-1] + 'RL' 
                    else:
                        # player 2 has chosen L
                        path[i][j]=  path[i][j - 2]  +'RR'
            # if i==j it means that there is only one mohreh
            else:
                path[i][j] = 'L'
    return table[0][n - 1], path[0][n - 1]

In [256]:
a = [ 10, 80, 90, 30]
print(find_max(a))

(110, 'RRRL')


# Q2) in my pdf solution

# Q3) M cities N roads -- Sum of all distances

### Description of solution is provided in my pdf solution

In [238]:
def add_binary_strings(a, b):
    '''
        add two binary strings without converting them to mabnaye 10 :)
    '''
    n = max(len(a), len(b))
    a = a.zfill(n)
    b = b.zfill(n)
    
    carry = 0
    result = []
    
    # begining from the end
    for i in range(n-1, -1, -1):
        bit_sum = int(a[i]) + int(b[i]) + carry
        bit = bit_sum % 2
        carry = bit_sum // 2
        result.append(str(bit))

    # if there is a carry add it
    if carry:
        result.append(str(carry))
    
    # make the order correct
    result.reverse()
    
    return ''.join(result)


def min_binary_string(a, b):
    '''
        compare two binary string and return the minimum
    '''
    n = max(len(a), len(b))
    # make the length the same
    a = a.zfill(n)
    b = b.zfill(n)
    # look for the first difference
    for i in range(n):
        if a[i] != b[i]:
            return a if a[i] < b[i] else b

    return a  # If both strings are equal, return either one


In [265]:
# Floyd Algorithm 
def floydw(dist):
    '''
    Floydwarshal algorithm
    
    I consulted this code : https://favtutor.com/blogs/floyd-warshall-algorithm
    
    '''
    V_size = len(dist)
    # Adding vertices individually
    for k in range(V_size):
        for p in range(V_size):
            for q in range(V_size):
                dist[p][q] = min_binary_string(dist[p][q], add_binary_strings(dist[p][k] , dist[k][q]))
    # this returns the matrix of distances in binary string format
    return dist

def sum_min_floydw(dis):

    # we only need the upper side of matrix diagonal so take this list for summation
    final_list = list(itertools.chain(*[[c for j,c in enumerate(l) if j>i] for i,l in enumerate(floyd(dis))]))

    # we add all the pairwise distances in binary
    S = ''
    for x in final_list:
        S = add_binary_strings (x,S)

    # remove the extra zeros
    return S.lstrip('0')

In [235]:
import itertools


def main():
    num_nodes, num_edges = map(int, input("Enter the number of cities and number of roads: ").split())
    
    #(Max 2* 10^5) in binary
    INF = '110000110101000000' 
    
    # matrix of result firts init by max distance 2* 10^5
    dist_mat = [[INF]*num_nodes for i in range(num_nodes)]
    
    # every node is connected to itself 
    for i in range(len(dist_mat)):
        dist_mat[i][i] = '0'
    
    # getting the input
    print("Enter the roads as 'node1 node2 distance_as_exponent_of_2':")
    for i in range(num_edges):
        node1, node2, exp = map(int, input(F'Enter the road {i}: ').split())
        # use the binary representation
        dist_mat[node1-1][node2-1] = '1'+'0'*exp
        dist_mat[node2-1][node1-1] = '1'+'0'*exp
    return dist_mat

In [236]:
mat = main()

Enter the number of cities and number of roads: 5 6
Enter the roads as 'node1 node2 distance_as_exponent_of_2':
Enter the road:   01 3 5
Enter the road:   14 5 0
Enter the road:   22 1 3
Enter the road:   33 2 1
Enter the road:   44 3 4
Enter the road:   54 2 2


In [266]:
sum_min_dis(mat)

'1000100'